#### Author: Roberto Mendoza
#### Analyzing RCT reemployment experiment

Data
In this lab, we analyze the Pennsylvania re-employment bonus experiment, which was previously studied in "Sequential testing of duration data: the case of the Pennsylvania ‘reemployment bonus’ experiment" (Bilias, 2000), among others. These experiments were conducted in the 1980s by the U.S. Department of Labor to test the incentive effects of alternative compensation schemes for unemployment insurance (UI).

In these experiments, UI claimants were randomly assigned either to a control group or one of five treatment groups. Actually, there are six treatment groups in the experiments. Here we focus on treatment group 4, but feel free to explore other treatment groups. In the control group the current rules of the UI applied. Individuals in the treatment groups were offered a cash bonus if they found a job within some pre-specified period of time (qualification period), provided that the job was retained for a specified duration. The treatments differed in the level of the bonus, the length of the qualification period, and whether the bonus was declining over time in the qualification period; see http://qed.econ.queensu.ca/jae/2000-v15.6/bilias/readme.b.txt for further details on data.

In [1]:
#import Pkg


#Pkg.add("DataFrames")
#Pkg.add("FilePaths")
#Pkg.add("Queryverse")
#Pkg.add("GLM")
#Pkg.add("StatsModels")
#Pkg.add("Combinatorics")
#Pkg.add("Iterators")
#Pkg.add("CategoricalArrays")
#Pkg.add("StatsBase")
#Pkg.add("Lasso")
#Pkg.add("TypedTables")
#Pkg.add("MacroTools")
#Pkg.add("NamedArrays")
#Pkg.add("DataTables")
#Pkg.add("Latexify")
#Pkg.add("PrettyTables")
#Pkg.add("TypedTables")
#Pkg.add("TexTables")
#Pkg.add("StatsModels")
#Pkg.add("DataTables")
#Pkg.add("FilePaths")
#Pkg.add("Combinatorics")
#Pkg.add("CategoricalArrays")
#Pkg.add("TypedTables")
#Pkg.add("MacroTools")

using GLM, StatsModels
using DataTables
using DelimitedFiles, DataFrames, Lasso
using FilePaths
using StatsModels, Combinatorics
using CategoricalArrays
using StatsBase, Statistics
using TypedTables
using MacroTools
using NamedArrays
using PrettyTables # Dataframe or Datatable to latex
using TexTables # pretty regression table and tex outcome

In [2]:
# Loading data

mat, head = readdlm("../../../data/penn_jae.dat", header=true, Float64)
mat
df =DataFrame(mat, vec(head))
describe(df)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,abdt,10693.6,10404.0,10691.0,10880.0,0,Float64
2,tg,2.56889,0.0,2.0,6.0,0,Float64
3,inuidur1,12.9148,1.0,10.0,52.0,0,Float64
4,inuidur2,12.1938,0.0,9.0,52.0,0,Float64
5,female,0.402142,0.0,0.0,1.0,0,Float64
6,black,0.116653,0.0,0.0,1.0,0,Float64
7,hispanic,0.0363689,0.0,0.0,1.0,0,Float64
8,othrace,0.00575002,0.0,0.0,1.0,0,Float64
9,dep,0.444045,0.0,0.0,2.0,0,Float64


In [179]:
#dimenntions of dataframe 

a = size(df,1)
b =  size(df,2)

23

In [3]:
# Filter control group and just treatment group number 4

penn = filter(row -> row[:tg] in [4,0], df)

first(penn,20)

,abdt,tg,inuidur1,inuidur2,female,black,hispanic,othrace,dep
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,10824.0,0.0,18.0,18.0,0.0,0.0,0.0,0.0,2.0
2,10824.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,10747.0,0.0,27.0,27.0,0.0,0.0,0.0,0.0,0.0
4,10607.0,4.0,9.0,9.0,0.0,0.0,0.0,0.0,0.0
5,10831.0,0.0,27.0,27.0,0.0,0.0,0.0,0.0,1.0
6,10845.0,0.0,27.0,27.0,1.0,0.0,0.0,0.0,0.0
7,10831.0,0.0,9.0,9.0,1.0,0.0,0.0,0.0,1.0
8,10859.0,0.0,27.0,27.0,1.0,0.0,0.0,0.0,1.0
9,10516.0,0.0,15.0,15.0,1.0,0.0,0.0,0.0,0.0


In [6]:
# Treatment group n°4
replace!(penn.tg, 4 => 1)


rename!(penn, "tg" => "T4")


# from float to string
penn[!,:dep] = string.(penn[!,:dep]) 

# dep varaible in categorical format 
penn[!,:dep] = categorical(penn[!,:dep])

describe(penn)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,abdt,10695.4,10404.0,10698.0,10880.0,0,Float64
2,T4,0.342224,0.0,0.0,1.0,0,Float64
3,inuidur1,13.053,1.0,11.0,52.0,0,Float64
4,inuidur2,12.2812,0.0,10.0,52.0,0,Float64
5,female,0.404001,0.0,0.0,1.0,0,Float64
6,black,0.121985,0.0,0.0,1.0,0,Float64
7,hispanic,0.0325554,0.0,0.0,1.0,0,Float64
8,othrace,0.00725632,0.0,0.0,1.0,0,Float64
9,dep,,0.0,,2.0,0,"CategoricalValue{String, UInt32}"


### 2.0 Carry out covariate balance check

In [7]:
    # couples variables combinations 
    combinations_upto(x, n) = Iterators.flatten(combinations(x, i) for i in 1:n)

    # combinations without same couple
    expand_exp(args, deg::ConstantTerm) =
        tuple(((&)(terms...) for terms in combinations_upto(args, deg.n))...)

    StatsModels.apply_schema(t::FunctionTerm{typeof(^)}, sch::StatsModels.Schema, ctx::Type) =
        apply_schema.(expand_exp(t.args_parsed...), Ref(sch), ctx)

In [8]:
# linear regression

reg1 = @formula(T4 ~ (female+black+othrace+dep+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)^2)
reg1 = apply_schema(reg1, schema(reg1, penn))

FormulaTerm
Response:
  T4(continuous)
Predictors:
  female(continuous)
  black(continuous)
  othrace(continuous)
  dep(DummyCoding:3→2)
  q2(continuous)
  q3(continuous)
  q4(continuous)
  q5(continuous)
  q6(continuous)
  agelt35(continuous)
  agegt54(continuous)
  durable(continuous)
  lusd(continuous)
  husd(continuous)
  female(continuous) & black(continuous)
  female(continuous) & othrace(continuous)
  female(continuous) & dep(DummyCoding:3→2)
  female(continuous) & q2(continuous)
  female(continuous) & q3(continuous)
  female(continuous) & q4(continuous)
  female(continuous) & q5(continuous)
  female(continuous) & q6(continuous)
  female(continuous) & agelt35(continuous)
  female(continuous) & agegt54(continuous)
  female(continuous) & durable(continuous)
  female(continuous) & lusd(continuous)
  female(continuous) & husd(continuous)
  black(continuous) & othrace(continuous)
  black(continuous) & dep(DummyCoding:3→2)
  black(continuous) & q2(continuous)
  black(continuous) & q3(

In [184]:
m1 = lm(reg1, penn)
table = regtable( "Covariate Balance Check" => m1) # coeficientes, standar error, squared R, N (sample size )

                   | Covariate Balance Check 
                   |           (1)           
---------------------------------------------
       (Intercept) |                  0.321* 
                   |                 (0.167) 
            female |                   0.104 
                   |                 (0.138) 
             black |                   0.072 
                   |                 (0.087) 
           othrace |                  -0.345 
                   |                 (0.294) 
          dep: 1.0 |                  -0.074 
                   |                 (0.218) 
          dep: 2.0 |                  -0.109 
                   |                 (0.165) 
                q2 |                  -0.027 
                   |                 (0.168) 
                q3 |                  -0.006 
                   |                 (0.167) 
                q4 |                   0.043 
                   |                 (0.168) 
                q5 |              

In [185]:
m1

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

T4 ~ 1 + female + black + othrace + dep + q2 + q3 + q4 + q5 + q6 + agelt35 + agegt54 + durable + lusd + husd + female & black + female & othrace + female & dep + female & q2 + female & q3 + female & q4 + female & q5 + female & q6 + female & agelt35 + female & agegt54 + female & durable + female & lusd + female & husd + black & othrace + black & dep + black & q2 + black & q3 + black & q4 + black & q5 + black & q6 + black & agelt35 + black & agegt54 + black & durable + black & lusd + black & husd + othrace & dep + othrace & q2 + othrace & q3 + othrace & q4 + othrace & q5 + othrace & q6 + othrace & agelt35 + othrace & agegt54 + othrace & durable + othrace & lusd + othrace & husd + dep & q2 + dep & q3 + dep & q4 + dep & q5 + dep & q6 + dep & agelt35 + dep & agegt54 + dep & durable + dep & lusd + dep & husd + q2 & q3 + q2 & q4 + 

### 3.0 Model specification

In [186]:
# No adjustment (2-sample approach)

ols_cl = lm(@formula(log(inuidur1) ~ T4), penn)

table1 = regtable( "No adjustment model" => ols_cl)   # 

            | No adjustment model 
            |         (1)         
----------------------------------
(Intercept) |            2.057*** 
            |             (0.021) 
         T4 |            -0.085** 
            |             (0.036) 
----------------------------------
          N |                5099 
      $R^2$ |               0.001 


In [187]:
# adding controls
# Omitted dummies: q1, nondurable, muld

reg2 = @formula(log(inuidur1) ~ T4 + (female+black+othrace+dep+q2+q3+q4+q5+q6+agelt35+agegt54+durable+lusd+husd)^2)
reg2 = apply_schema(reg2, schema(reg2, penn))

ols_cra = lm(reg2, penn)
table2 = regtable("CRA model" => ols_cra)

                   | CRA model 
                   |    (1)    
-------------------------------
       (Intercept) |  2.633*** 
                   |   (0.420) 
                T4 |  -0.080** 
                   |   (0.036) 
            female |    -0.115 
                   |   (0.347) 
             black | -0.441*** 
                   |   (0.158) 
           othrace |    -0.883 
                   |   (0.903) 
          dep: 1.0 |    -0.720 
                   |   (0.550) 
          dep: 2.0 |    -0.041 
                   |   (0.417) 
                q2 |    -0.160 
                   |   (0.423) 
                q3 |    -0.540 
                   |   (0.422) 
                q4 |    -0.433 
                   |   (0.422) 
                q5 |    -0.345 
                   |   (0.420) 
                q6 |    -0.494 
                   |   (0.420) 
           agelt35 |   -0.626* 
                   |   (0.340) 
           agegt54 |    -0.361 
                   |   (0.760) 
        

In [188]:
# demean function
function desv_mean(a)
    A = mean(a, dims = 1)
    M = zeros(Float64, size(X,1), size(X,2))
    
    for i in 1:size(a,2)
          M[:,i] = a[:,i] .- A[i]
    end
    return M
end    



# Matrix Model & demean
X = StatsModels.modelmatrix(reg1.rhs,penn)
X = desv_mean(X) # matrix format 
 


5099×119 Matrix{Float64}:
 -0.404001  -0.121985  -0.00725632  -0.112179  …  -0.0543244  -0.0280447  0.0
 -0.404001  -0.121985  -0.00725632  -0.112179     -0.0543244  -0.0280447  0.0
 -0.404001  -0.121985  -0.00725632  -0.112179     -0.0543244  -0.0280447  0.0
 -0.404001  -0.121985  -0.00725632  -0.112179     -0.0543244  -0.0280447  0.0
 -0.404001  -0.121985  -0.00725632   0.887821      0.945676   -0.0280447  0.0
  0.595999  -0.121985  -0.00725632  -0.112179  …  -0.0543244  -0.0280447  0.0
  0.595999  -0.121985  -0.00725632   0.887821     -0.0543244  -0.0280447  0.0
  0.595999  -0.121985  -0.00725632   0.887821     -0.0543244  -0.0280447  0.0
  0.595999  -0.121985  -0.00725632  -0.112179     -0.0543244  -0.0280447  0.0
  0.595999  -0.121985  -0.00725632  -0.112179     -0.0543244  -0.0280447  0.0
  0.595999  -0.121985  -0.00725632  -0.112179  …  -0.0543244  -0.0280447  0.0
  0.595999  -0.121985  -0.00725632  -0.112179     -0.0543244  -0.0280447  0.0
  0.595999  -0.121985  -0.00725632  -0

In [189]:
Y = select(penn, [:inuidur1,:T4]) # select inuidur1 y T4

X = DataFrame(hcat(X, Matrix(select(penn, [:T4])).*X), :auto)  # Joint X, (T4*X)

base = hcat(Y, X) # Joint inuidur1, T4, X y (T4*X)

base.inuidur1 = log.(base.inuidur1)  # log(inuidur1)

terms = term.(names(base)) # term.() let us to get all variables as objects

#interactive regression model

ols_ira  = lm(terms[1] ~ sum(terms[2:end]), base)


table3 = regtable("Interactive model" => ols_ira)

#terms[1] : select first variable. In this case, oucome of interest 
#sum(terms[2:end]) : independent variables as regresors in the linear regression   

            | Interactive model 
            |        (1)        
--------------------------------
(Intercept) |          2.058*** 
            |           (0.021) 
         T4 |          -0.076** 
            |           (0.036) 
         x1 |            -0.666 
            |           (0.443) 
         x2 |          -0.437** 
            |           (0.196) 
         x3 |            -1.735 
            |           (2.163) 
         x4 |             0.036 
            |           (0.682) 
         x5 |             0.212 
            |           (0.495) 
         x6 |            -0.255 
            |           (0.525) 
         x7 |            -0.621 
            |           (0.523) 
         x8 |            -0.480 
            |           (0.524) 
         x9 |            -0.372 
            |           (0.522) 
        x10 |            -0.677 
            |           (0.519) 
        x11 |            -0.678 
            |           (0.433) 
        x12 |            -0.304 
          

In [208]:
X = StatsModels.modelmatrix(reg2.rhs,penn)
X = desv_mean(X)


D = DataFrame([X[:,1]], :auto)  # Treatment varaible

rename!(D, Dict(:x1 => :T4)) #rename x1 -> T4

X = DataFrame(hcat(X[:,2:end], X[:,1].*X[:,2:end]), :auto)  # Join Controls (X) + T4*X "interactive"

Y = select(penn, [:inuidur1]) #select just inuidur1

Y.inuidur1 = log.(Y.inuidur1)  # log(inuidur1)





#terms = term.(names(base)) # all terms  
#model = terms[1] ~ sum(terms[2:end])
#lazso = fit(LassoModel,terms[1] ~ sum(terms[2:end]), base; standardize=false, α = 0.1)
#coef(lasso)[2]
#table3 = regtable("Lasso adjustment model" => lasso)

5099-element Vector{Float64}:
 2.8903717578961645
 0.0
 3.295836866004329
 2.1972245773362196
 3.295836866004329
 3.295836866004329
 2.1972245773362196
 3.295836866004329
 2.70805020110221
 3.332204510175204
 2.4849066497880004
 3.091042453358316
 2.8903717578961645
 ⋮
 3.295836866004329
 2.70805020110221
 2.995732273553991
 0.0
 3.1354942159291497
 2.5649493574615367
 1.791759469228055
 2.302585092994046
 1.3862943611198906
 2.1972245773362196
 1.3862943611198906
 3.295836866004329

In [220]:
X

,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-0.404001,-0.121985,-0.00725632,-0.112179,0.836242,-0.203765,-0.235536,-0.225927
2,-0.404001,-0.121985,-0.00725632,-0.112179,-0.163758,-0.203765,-0.235536,-0.225927
3,-0.404001,-0.121985,-0.00725632,-0.112179,-0.163758,-0.203765,-0.235536,0.774073
4,-0.404001,-0.121985,-0.00725632,-0.112179,-0.163758,-0.203765,0.764464,-0.225927
5,-0.404001,-0.121985,-0.00725632,0.887821,-0.163758,-0.203765,-0.235536,-0.225927
6,0.595999,-0.121985,-0.00725632,-0.112179,-0.163758,-0.203765,-0.235536,-0.225927
7,0.595999,-0.121985,-0.00725632,0.887821,-0.163758,-0.203765,-0.235536,-0.225927
8,0.595999,-0.121985,-0.00725632,0.887821,-0.163758,-0.203765,-0.235536,-0.225927
9,0.595999,-0.121985,-0.00725632,-0.112179,-0.163758,0.796235,-0.235536,-0.225927


# Using HDMJL

In [221]:
names(X)

238-element Vector{String}:
 "x1"
 "x2"
 "x3"
 "x4"
 "x5"
 "x6"
 "x7"
 "x8"
 "x9"
 "x10"
 "x11"
 "x12"
 "x13"
 ⋮
 "x227"
 "x228"
 "x229"
 "x230"
 "x231"
 "x232"
 "x233"
 "x234"
 "x235"
 "x236"
 "x237"
 "x238"

In [265]:
include("../hdmjl/hdmjl.jl")

In [267]:
D_reg_0  = rlasso_arg( X, D, nothing, true, true, true, false, false, 
                    nothing, 1.1, nothing, 5000, 15, 10^(-5), -Inf, true, Inf, true )

rlasso_arg(5099×238 DataFrame
  Row │ x1         x2         x3           x4         x5         x6         x7 ⋯
      │ Float64    Float64    Float64      Float64    Float64    Float64    Fl ⋯
──────┼─────────────────────────────────────────────────────────────────────────
    1 │ -0.404001  -0.121985  -0.00725632  -0.112179   0.836242  -0.203765  -0 ⋯
    2 │ -0.404001  -0.121985  -0.00725632  -0.112179  -0.163758  -0.203765  -0
    3 │ -0.404001  -0.121985  -0.00725632  -0.112179  -0.163758  -0.203765  -0
    4 │ -0.404001  -0.121985  -0.00725632  -0.112179  -0.163758  -0.203765   0
    5 │ -0.404001  -0.121985  -0.00725632   0.887821  -0.163758  -0.203765  -0 ⋯
    6 │  0.595999  -0.121985  -0.00725632  -0.112179  -0.163758  -0.203765  -0
    7 │  0.595999  -0.121985  -0.00725632   0.887821  -0.163758  -0.203765  -0
    8 │  0.595999  -0.121985  -0.00725632   0.887821  -0.163758  -0.203765  -0
    9 │  0.595999  -0.121985  -0.00725632  -0.112179  -0.163758   0.796235  -0 ⋯
   10 │  0

In [268]:
D_resid = rlasso(D_reg_0)

Dict{String, Any} with 19 entries:
  "tss"          => 1147.82
  "dev"          => [-0.342224, -0.342224, -0.342224, 0.657776, -0.342224, -0.3…
  "model"        => [-0.404001 -0.121985 … 0.0101738 0.0; -0.404001 -0.121985 ……
  "loadings"     => [0.232712 0.155595 … 0.0435631 0.0]
  "sigma"        => [0.474501]
  "lambda0"      => 637.701
  "lambda"       => 238×2 DataFrame…
  "intercept"    => 1.82896e-17
  "Xy"           => [-3.98137, 2.13669, 5.33771, -1.75211, 3.24299, -3.5707, -4…
  "iter"         => 4
  "residuals"    => [-0.342224, -0.342224, -0.342224, 0.657776, -0.342224, -0.3…
  "rss"          => 1147.82
  "index"        => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, …
  "beta"         => 238×2 DataFrame…
  "options"      => Dict{String, Any}("intercept"=>true, "post"=>true, "meanx"=…
  "x1"           => Matrix{Float64}(undef, 5099, 0)
  "pen"          => Dict{String, Any}("lambda0"=>637.701, "lambda"=>[148.401; 9…
  "startingval"  => [-0.342224, -0.342224, -0.3

In [269]:
D_resid = rlasso(D_reg_0)["residuals"]

5099-element Vector{Float64}:
 -0.34222396548342815
 -0.34222396548342815
 -0.34222396548342815
  0.6577760345165719
 -0.34222396548342815
 -0.34222396548342815
 -0.34222396548342815
 -0.34222396548342815
 -0.34222396548342815
 -0.34222396548342815
 -0.34222396548342815
  0.6577760345165719
 -0.34222396548342815
  ⋮
 -0.34222396548342815
 -0.34222396548342815
  0.6577760345165719
 -0.34222396548342815
 -0.34222396548342815
  0.6577760345165719
 -0.34222396548342815
  0.6577760345165719
  0.6577760345165719
 -0.34222396548342815
  0.6577760345165719
 -0.34222396548342815

In [270]:
Y_reg_0  = rlasso_arg( X, Y, nothing, true, true, true, false, false, 
                    nothing, 1.1, nothing, 5000, 15, 10^(-5), -Inf, true, Inf, true )


rlasso_arg(5099×238 DataFrame
  Row │ x1         x2         x3           x4         x5         x6         x7 ⋯
      │ Float64    Float64    Float64      Float64    Float64    Float64    Fl ⋯
──────┼─────────────────────────────────────────────────────────────────────────
    1 │ -0.404001  -0.121985  -0.00725632  -0.112179   0.836242  -0.203765  -0 ⋯
    2 │ -0.404001  -0.121985  -0.00725632  -0.112179  -0.163758  -0.203765  -0
    3 │ -0.404001  -0.121985  -0.00725632  -0.112179  -0.163758  -0.203765  -0
    4 │ -0.404001  -0.121985  -0.00725632  -0.112179  -0.163758  -0.203765   0
    5 │ -0.404001  -0.121985  -0.00725632   0.887821  -0.163758  -0.203765  -0 ⋯
    6 │  0.595999  -0.121985  -0.00725632  -0.112179  -0.163758  -0.203765  -0
    7 │  0.595999  -0.121985  -0.00725632   0.887821  -0.163758  -0.203765  -0
    8 │  0.595999  -0.121985  -0.00725632   0.887821  -0.163758  -0.203765  -0
    9 │  0.595999  -0.121985  -0.00725632  -0.112179  -0.163758   0.796235  -0 ⋯
   10 │  0

In [271]:
Y_resid = rlasso(Y_reg_0)["residuals"]

5099-element Vector{Float64}:
  0.7878744641425394
 -2.102497293753625
  1.193339572250704
  0.23664761558313163
  0.9880047917342458
  0.9880047917342458
 -0.11060749693386379
  1.193339572250704
  0.6055529073485849
  1.0243724359051205
  0.5243296880349124
  1.1304654916052281
  0.5825396836260812
  ⋮
  0.17144757386479892
 -0.27441875903678303
  1.035155311800903
 -1.960576961753088
  1.0329969221755246
  0.9080172149314624
 -0.31073782452557014
  0.34200813124095797
 -0.7162029326337346
  0.23664761558313163
 -0.9215377131501927
  0.9880047917342458

In [278]:
D_resid = reshape(D_resid, length(D_resid), 1)

5099×1 Matrix{Float64}:
 -0.34222396548342815
 -0.34222396548342815
 -0.34222396548342815
  0.6577760345165719
 -0.34222396548342815
 -0.34222396548342815
 -0.34222396548342815
 -0.34222396548342815
 -0.34222396548342815
 -0.34222396548342815
 -0.34222396548342815
  0.6577760345165719
 -0.34222396548342815
  ⋮
 -0.34222396548342815
 -0.34222396548342815
  0.6577760345165719
 -0.34222396548342815
 -0.34222396548342815
  0.6577760345165719
 -0.34222396548342815
  0.6577760345165719
  0.6577760345165719
 -0.34222396548342815
  0.6577760345165719
 -0.34222396548342815

In [279]:
Lasso_ira = lm(D_resid, Y_resid)

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}:

Coefficients:
───────────────────────────────────────────────────────────────────
         Coef.  Std. Error      t  Pr(>|t|)  Lower 95%    Upper 95%
───────────────────────────────────────────────────────────────────
x1  -0.0788861   0.0355478  -2.22    0.0265  -0.148575  -0.00919709
───────────────────────────────────────────────────────────────────


In [285]:

table = NamedArray(zeros(4, 5))

4×5 Named Matrix{Float64}
A ╲ B │   1    2    3    4    5
──────┼────────────────────────
1     │ 0.0  0.0  0.0  0.0  0.0
2     │ 0.0  0.0  0.0  0.0  0.0
3     │ 0.0  0.0  0.0  0.0  0.0
4     │ 0.0  0.0  0.0  0.0  0.0

In [323]:
# Comparative ATE estimation

table = NamedArray(zeros(4, 5))

table[1,2] = GLM.coeftable(ols_cl).cols[1][2]
table[2,2] = GLM.coeftable(ols_cl).cols[2][2]
table[3,2] = GLM.coeftable(ols_cl).cols[5][2]
table[4,2] = GLM.coeftable(ols_cl).cols[6][2]
table[1,3] = GLM.coeftable(ols_cra).cols[1][2]
table[2,3] = GLM.coeftable(ols_cra).cols[2][2]
table[3,3] = GLM.coeftable(ols_cra).cols[5][2]
table[4,3] = GLM.coeftable(ols_cra).cols[6][2]
table[1,4] = GLM.coeftable(ols_ira).cols[1][2]
table[2,4] = GLM.coeftable(ols_ira).cols[2][2]
table[3,4] = GLM.coeftable(ols_ira).cols[5][2]
table[4,4] = GLM.coeftable(ols_ira).cols[6][2]
table[1,5] = GLM.coeftable(Lasso_ira).cols[1][1]
table[2,5] = GLM.coeftable(Lasso_ira).cols[2][1]
table[3,5] = GLM.coeftable(Lasso_ira).cols[5][1]
table[4,5] = GLM.coeftable(Lasso_ira).cols[6][1]

T = DataFrame(table, [ :"Outcome", :"CL", :"CRA", :"IRA", :"IRA W Lasso"])  # table to dataframe 
T[!,:Outcome] = string.(T[!,:Outcome])  # string - first column 

T[1,1] = "Estimation"
T[2,1] = "Standar error"
T[3,1] = "Lower bound CI"
T[4,1] = "Upper bound CI"

header = (["Outcome", "CL", "CRA", "IRA", "IRA W Lasso"])

pretty_table(T; backend = Val(:html), header = header, formatters=ft_round(4), alignment=:c)

#Val(:latex) to tex format (Dataframe or DataTable, table , etc)

#pretty_table(T; backend = Val(:latex), header = header, formatters=ft_round(4), alignment=:c)


Outcome,CL,CRA,IRA,IRA W Lasso
Estimation,-0.0855,-0.0797,-0.0755,-0.0789
Standar error,0.0358,0.0356,0.0361,0.0355
Lower bound CI,-0.1557,-0.1496,-0.1462,-0.1486
Upper bound CI,-0.0152,-0.0098,-0.0048,-0.0092
